<h1>Extraite données de l'url</h1>

In [3]:
import re
import pandas as pd
import numpy as np

train_1 = pd.read_csv('../Data/train_1.csv')
train_1

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145058,Underworld_(serie_de_películas)_es.wikipedia.o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13.0,12.0,13.0,3.0,5.0,10.0
145059,Resident_Evil:_Capítulo_Final_es.wikipedia.org...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145060,Enamorándome_de_Ramón_es.wikipedia.org_all-acc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145061,Hasta_el_último_hombre_es.wikipedia.org_all-ac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Solutions trouvées ici https://github.com/Arturus/kaggle-web-traffic/blob/master/extractor.py

import re
import pandas as pd
import numpy as np

term_pat = re.compile('(.+?):(.+)')
pat = re.compile(
    '(.+)_([a-z][a-z]\.)?((?:wikipedia\.org)|(?:commons\.wikimedia\.org)|(?:www\.mediawiki\.org))_([a-z_-]+?)$')

# Debug output to ensure pattern still works
# print(pat.fullmatch('BLEACH_zh.wikipedia.org_all-accessspider').groups())
# print(pat.fullmatch('Accueil_commons.wikimedia.org_all-access_spider').groups())


def extract(source) -> pd.DataFrame:
    """
    Extracts features from url. Features: agent, site, country, term, marker
    :param source: urls
    :return: DataFrame, one column per feature
    """
    if isinstance(source, pd.Series):
        source = source.values
    agents = np.full_like(source, np.NaN)
    sites = np.full_like(source, np.NaN)
    countries = np.full_like(source, np.NaN)
    terms = np.full_like(source, np.NaN)
    markers = np.full_like(source, np.NaN)

    for i in range(len(source)):
        l = source[i]
        match = pat.fullmatch(l)
        assert match, "Non-matched string %s" % l
        term = match.group(1)
        country = match.group(2)
        if country:
            countries[i] = country[:-1]
        site = match.group(3)
        sites[i] = site
        agents[i] = match.group(4)
        if site != 'wikipedia.org':
            term_match = term_pat.match(term)
            if term_match:
                markers[i] = term_match.group(1)
                term = term_match.group(2)
        terms[i] = term

    return pd.DataFrame({
        'agent': agents,
        'site': sites,
        'country': countries,
        'term': terms,
        'marker': markers,
        'page': source
    })

In [15]:
extract_df = extract(train_1['Page'])
print(extract_df.head())

               agent           site country              term marker  \
0  all-access_spider  wikipedia.org      zh              2NE1    NaN   
1  all-access_spider  wikipedia.org      zh               2PM    NaN   
2  all-access_spider  wikipedia.org      zh                3C    NaN   
3  all-access_spider  wikipedia.org      zh           4minute    NaN   
4  all-access_spider  wikipedia.org      zh  52_Hz_I_Love_You    NaN   

                                                page  
0            2NE1_zh.wikipedia.org_all-access_spider  
1             2PM_zh.wikipedia.org_all-access_spider  
2              3C_zh.wikipedia.org_all-access_spider  
3         4minute_zh.wikipedia.org_all-access_spider  
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...  


In [19]:
for column in extract_df.columns:
    print(str(column), pd.unique(extract_df[column]))

agent ['all-access_spider' 'desktop_all-agents' 'mobile-web_all-agents'
 'all-access_all-agents']
site ['wikipedia.org' 'commons.wikimedia.org' 'www.mediawiki.org']
country ['zh' 'fr' 'en' nan 'ru' 'de' 'ja' 'es']
term ['2NE1' '2PM' '3C' ... 'Eine_Reihe_betrüblicher_Ereignisse_(Fernsehserie)'
 'ジャック・リーチャー' 'マイク眞木']
marker [nan 'Campaign' 'Category' 'Commons' 'File' 'Help' 'Image' 'MediaWiki'
 'Special' 'User' 'User_talk' 'Talk' 'special' 'Template' 'API' 'API_talk'
 'Extension' 'Extension_talk' 'Help_talk' 'Manual' 'Manual_talk' 'Project'
 'Skin' 'Thread' 'Topic' 'Project_talk' 'Template_talk' 'VisualEditor'
 'help' 'Category_talk' 'Toolserver' 'File_talk' 'Translations_talk'
 'category' 'Commons_talk']
page ['2NE1_zh.wikipedia.org_all-access_spider'
 '2PM_zh.wikipedia.org_all-access_spider'
 '3C_zh.wikipedia.org_all-access_spider' ...
 'Enamorándome_de_Ramón_es.wikipedia.org_all-access_spider'
 'Hasta_el_último_hombre_es.wikipedia.org_all-access_spider'
 'Francisco_el_matemático_(seri

In [1]:
from extract import extract_url

In [4]:
extract_url(train_1['Page'])

,agent,site,country,term,marker,page
0,all-access_spider,wikipedia.org,zh,2NE1,NaN,2NE1_zh.wikipedia.org_all-access_spider
1,all-access_spider,wikipedia.org,zh,2PM,NaN,2PM_zh.wikipedia.org_all-access_spider
2,all-access_spider,wikipedia.org,zh,3C,NaN,3C_zh.wikipedia.org_all-access_spider
3,all-access_spider,wikipedia.org,zh,4minute,NaN,4minute_zh.wikipedia.org_all-access_spider
4,all-access_spider,wikipedia.org,zh,52_Hz_I_Love_You,NaN,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
...,...,...,...,...,...,...
145058,all-access_spider,wikipedia.org,es,Underworld_(serie_de_películas),NaN,Underworld_(serie_de_películas)_es.wikipedia.o...
145059,all-access_spider,wikipedia.org,es,Resident_Evil:_Capítulo_Final,NaN,Resident_Evil:_Capítulo_Final_es.wikipedia.org...
145060,all-access_spider,wikipedia.org,es,Enamorándome_de_Ramón,NaN,Enamorándome_de_Ramón_es.wikipedia.org_all-acc...
145061,all-access_spider,wikipedia.org,es,Hasta_el_último_hombre,NaN,Hasta_el_último_hombre_es.wikipedia.org_all-ac...
